In [1]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin
import json, requests, pickle

import smtplib
import os
from email.mime.text import MIMEText
from email import encoders

In [2]:
# Google 아이디, 비밀번호 저장 파일
EMAIL_USER_DATA_SAVED_FILE = 'res/account'

In [3]:
# 계정 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작하면(계정 정보가 다시 리셋)
import pickle

email_user_info = dict({
    "smtp_user_id": "wsk0307@gmail.com",
    "smtp_user_pw": ""
})

f = open(EMAIL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(email_user_info, f)
f.close()

In [4]:
f = open(EMAIL_USER_DATA_SAVED_FILE, 'rb')
email_user_info = pickle.load(f)
f.close()

In [5]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info['smtp_port']) as server:
        server.starttls()
        
        server.login(email_user_info['smtp_user_id'], email_user_info['smtp_user_pw'])
        
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공')
        
            return 'success'
        else:
            print(res)
 
            return 'fail'

In [6]:
import datetime

app = Flask(__name__)
CORS(app)

@app.route('/findPwByEmail', methods = ['GET', 'POST'])
def findPwByEmail():
    params = json.loads(request.get_data(), encoding='utf-8')
    
    name = params['name']
    email = params['email']
    
    print(name, email)

    smtp_info = dict({
        'smtp_server': 'smtp.gmail.com',
        'smtp_port': 587
    })

    now = datetime.datetime.now()
    now_time = now.strftime('%Y-%m-%d %H:%M')
    
    title = '유러너 - ' + name + '님 비밀번호 재설정 링크입니다.'
    content =  now_time + '에 비밀번호 재설정을 요청하였습니다.\n아래 링크에서 비밀번호 재설정을 완료해 주세요.\nhttp://http://localhost:8080/changepw?name=' + name 
    sender = email_user_info['smtp_user_id']
    receiver = email
    
    msg = MIMEText(_text = content, _charset = 'utf-8')

    msg['Subject'] = title
    msg['From'] = sender
    msg['To'] = receiver
    
    isSuccess = send_email(smtp_info, msg)

    print(isSuccess)
    
    return isSuccess

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-6-dc20eb1a86ee>:8: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')


daniel wsk0307@gmail.com


127.0.0.1 - - [28/Oct/2021 17:41:16] "POST /findPwByEmail HTTP/1.1" 200 -


이메일 전송 성공
success
